In [12]:
import pandas as pd
import plotly.express as px

from flights import rapidapi

from tqdm import tqdm_notebook as tqdm
rapidapi.tqdm = tqdm

In [13]:
df = rapidapi.query_pair("BCN", "CAG")

In [14]:
df.tail()

,MinPrice,Direct,QuoteDateTime,DepartureDate,Carrier,Origin,Destination,Inserted
,,,,,,,,
20,83.0,False,2020-01-04 17:40:00,2020-10-19,Iberia,BCN,CAG,2020-01-12
21,44.0,True,2020-01-08 23:03:00,2020-10-21,Vueling Airlines,BCN,CAG,2020-01-12
22,73.0,False,2020-01-05 12:52:00,2020-10-22,Ryanair,BCN,CAG,2020-01-12
23,32.0,True,2020-01-09 09:44:00,2020-10-23,Vueling Airlines,BCN,CAG,2020-01-12
24,64.0,False,2020-01-09 09:44:00,2020-10-24,Vueling Airlines,BCN,CAG,2020-01-12
